In [1]:
import torch
from torch import nn
import numpy as np

In [2]:
Example ='''The sun rose slowly over the quiet hills.
    She placed her backpack by the door and sat down.
    A cool breeze moved through the open window.
    They finished their homework before dinner.
    The dog waited patiently for its owner to return.
    Rain tapped softly against the glass.
    He wrote a reminder note and stuck it to the fridge.
    The library was silent except for turning pages.
    Fresh bread filled the kitchen with a warm smell.
    The train arrived a few minutes early.'''
Example = Example.split()


In [3]:
Vec_size = 10

In [4]:
words = set(Example)
words = list(words)
V = len(words)

In [5]:
v_center_vectors = np.random.randn(V, Vec_size) * 0.01
u_context_vectors = np.random.randn(V, Vec_size) * 0.01

In [6]:
def probability_given_pair(cwvi, covi): # center word vector index, context word vector index
    return u_context_vectors[covi].T @ v_center_vectors[cwvi] - np.log10(np.sum([np.exp(u_context_vectors[i].T @ v_center_vectors[covi]) for i in range(len(u_context_vectors))]))

In [7]:
def get_vectoridx_with_wordidx(word_idx):
    return words.index(Example[word_idx])

In [8]:
def log_likelyhood_loss(cwi, context_window):
    text_seq_sum = 0     
    for index in range(2*context_window + 1):
        if(index == cwi):
            continue
        coi = cwi + index - 2 
        text_seq_sum += probability_given_pair(get_vectoridx_with_wordidx(cwi), get_vectoridx_with_wordidx(coi))
    text_seq_sum *= -1 
    return text_seq_sum

In [9]:
def grad_center_vector(cwvi, covi):
    v_c = v_center_vectors[cwvi]          
    u_o = u_context_vectors[covi]        
    scores = u_context_vectors @ v_c       
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores)

    expected = np.sum(probs[:, np.newaxis] * u_context_vectors, axis=0)

    grad = u_o - expected
    return grad


In [11]:
def skipgram_sgd_step(cwvi, covi, lr=0.01):
    global v_center_vectors, u_context_vectors
    center_vector = v_center_vectors[cwvi]  
    context_vector = u_context_vectors[covi] 

    scores = u_context_vectors @ center_vector
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores) 

    grad_vc = context_vector - np.sum(probs[:, np.newaxis] * u_context_vectors, axis=0)

    grad_u = - probs[:, np.newaxis] * center_vector
    grad_u[covi] += center_vector 

    v_center_vectors[cwvi] += lr * grad_vc
    u_context_vectors += lr * grad_u


In [12]:
word2idx = {w: i for i, w in enumerate(words)}
corpus_indices = [word2idx[w] for w in Example]

In [14]:
window_size = 2
lr = 0.01

epochs = 10000
for i in range(epochs):
    for center_pos, center_word_idx in enumerate(corpus_indices):
        start = max(center_pos - window_size, 0)
        end = min(center_pos + window_size + 1, len(corpus_indices))
        
        for context_pos in range(start, end):
            if context_pos == center_pos:
                continue
            
            context_word_idx = corpus_indices[context_pos]
            
            skipgram_sgd_step(center_word_idx, context_word_idx, lr)


In [15]:
from numpy.linalg import norm

def nearest_neighbors(target_word, top_k=5):
    idx = word2idx[target_word]
    v_target = v_center_vectors[idx]
    sims = []
    for i, word in enumerate(words):
        if i == idx:
            continue
        v = v_center_vectors[i]
        cosine = np.dot(v_target, v) / (norm(v_target) * norm(v))
        sims.append((word, cosine))
    sims.sort(key=lambda x: x[1], reverse=True)
    return sims[:top_k]

In [16]:
nearest_neighbors("tapped")

[('Rain', np.float64(0.589378120380917)),
 ('softly', np.float64(0.5099769320302998)),
 ('finished', np.float64(0.3939599926283824)),
 ('against', np.float64(0.3719396352272689)),
 ('by', np.float64(0.3168856860460859))]

In [17]:
# The oringin word was 
# ...... Rain tapped softly ..........